# Testing monetization workflow with `.mif` and mapping files

In [80]:
from premise.data_collection import IAMDataCollection
from premise.activity_maps import InventorySet
import brightway2 as bw
import pandas as pd
import xarray as xr
import numpy as np

## 1. Load REMIND reporting file

As an example, we'll work with production volumes.

In [3]:
iam_data = IAMDataCollection("remind", "default", 2030, "../remind_data/", None)
iam_data.production_volumes

<xarray.DataArray 'value' (region: 13, variables: 48, year: 19)>
array([[[5.96300000e-04, 7.60100000e-04, 7.53900000e-04, ...,
         7.00000000e-07, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [4.68424000e-02, 5.46379000e-02, 5.40414000e-02, ...,
         8.00000000e-07, 0.00000000e+00, 0.00000000e+00],
        ...,
        [7.14992000e+00, 5.54636670e+00, 4.93317990e+00, ...,
         1.46478307e+01, 1.60704899e+01, 1.53692196e+01],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [4.74387000e-02, 5.53980000e-02, 5.47953000e-02, ...,
         1.28798100e-01, 1.28802100e-01, 1.28802100e-01]],

       [[0.00000000e+00, 1.77089000e-02, 9.65938000e-02, ...,
         2.00000000e-07, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [2.95090000e-02, 3.96727000e-02, 9.10959000e-02, ...,
         3.00000000e-07, 0.00000000e+00, 0.00000000e+00],
...
        [5.19730000e+01, 4.98313800e+01, 4.32051948e+01, ...,
         8.80261997e+01, 9.76852922e+01, 9.43328453e+01],
        [0.00000000e+00, 1.43913000e-02, 0.00000000e+00, ...,
         8.31141900e-01, 8.95337900e-01, 8.95337900e-01],
        [2.68090600e-01, 3.03268200e-01, 3.25662100e-01, ...,
         3.14828600e-01, 3.14828800e-01, 3.14828800e-01]],

       [[1.82647900e-01, 3.12584900e-01, 4.81587600e-01, ...,
         7.30000000e-06, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [6.63479500e-01, 8.81074600e-01, 1.14710380e+00, ...,
         7.40000000e-06, 0.00000000e+00, 0.00000000e+00],
        ...,
        [3.13925790e+02, 3.47858923e+02, 2.65295198e+02, ...,
         9.63537328e+02, 9.63594052e+02, 8.09446947e+02],
        [8.00000000e-07, 1.01057500e-01, 4.41331000e-02, ...,
         4.34117990e+00, 6.29010340e+00, 6.59171230e+00],
        [8.46126600e-01, 1.09260190e+00, 1.58455830e+00, ...,
         3.20421040e+00, 3.31786000e+00, 3.31786000e+00]]])
Coordinates:
  * region     (region) object 'CAZ' 'CHA' 'EUR' 'IND' ... 'SSA' 'USA' 'World'
  * variables  (variables) <U38 'Biomass CHP' ... 'biomass - residual'
  * year       (year) int64 2005 2010 2015 2020 2025 ... 2100 2110 2130 2150

## 2. Load database

In [4]:
from premise import NewDatabase
from premise.activity_maps import get_mapping
bw.projects.set_current("test")

In [5]:
ndb = NewDatabase(
    scenarios = [
        {"model":"remind", "pathway":"SSP2-Base", "year":2035},
    ],
    source_db="ecoinvent 3.7.1 cutoff",
    source_version="3.7.1",
    key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
)

premise v.(1, 3, 2)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

## 3. Get a mapping file

In [6]:
from premise.data_collection import IAM_ELEC_VARS as fp
mapping = get_mapping(fp, "iam_aliases")
premise2rmnd = {}
for k, v in mapping.items():
    if "remind" in v.keys():
        premise2rmnd[k] = v["remind"]
premise2rmnd

{'Biomass CHP': 'SE|Electricity|Biomass|++|Combined Heat and Power w/o CC',
 'Biomass IGCC CCS': 'SE|Electricity|Biomass|++|Gasification Combined Cycle w/ CC',
 'Biomass IGCC': 'SE|Electricity|Biomass|++|Gasification Combined Cycle w/o CC',
 'Coal PC': 'SE|Electricity|Coal|++|Pulverised Coal w/o CC',
 'Coal IGCC': 'SE|Electricity|Coal|++|Gasification Combined Cycle w/o CC',
 'Coal PC CCS': 'SE|Electricity|Coal|++|Pulverised Coal w/ CC',
 'Coal IGCC CCS': 'SE|Electricity|Coal|++|Gasification Combined Cycle w/ CC',
 'Coal CHP': 'SE|Electricity|Coal|++|Combined Heat and Power w/o CC',
 'Gas OC': 'SE|Electricity|Gas|++|Gas Turbine',
 'Gas CC': 'SE|Electricity|Gas|++|Combined Cycle w/o CC',
 'Gas CHP': 'SE|Electricity|Gas|++|Combined Heat and Power w/o CC',
 'Gas CC CCS': 'SE|Electricity|Gas|++|Combined Cycle w/ CC',
 'Geothermal': 'SE|Electricity|+|Geothermal',
 'Hydro': 'SE|Electricity|+|Hydro',
 'Nuclear': 'SE|Electricity|+|Nuclear',
 'Oil ST': 'SE|Electricity|Oil|w/o CC',
 'Solar CSP': 

In [7]:
InvSet = InventorySet(ndb.database)

In [8]:
mapping = InvSet.generate_powerplant_map()
mapping

{'Biomass CHP': {'heat and power co-generation, wood chips, 2000 kW',
  'heat and power co-generation, wood chips, 2000 kW, state-of-the-art 2014',
  'heat and power co-generation, wood chips, 6667 kW',
  'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014',
  'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014, label-certified'},
 'Biomass CHP CCS': {'electricity production, at co-generation power plant/wood, post, pipeline 200km, storage 1000m',
  'electricity production, at co-generation power plant/wood, post, pipeline 400km, storage 3000m'},
 'Biomass ST': {'electricity production, at wood burning power plant 20 MW, truck 25km, no CCS'},
 'Biomass IGCC CCS': {'electricity production, at BIGCC power plant, pre, pipeline 200km, storage 1000m',
  'electricity production, at wood burning power plant 20 MW, truck 25km, post, pipeline 200km, storage 1000m'},
 'Biomass IGCC': {'electricity production, at BIGCC power plant, no CCS'},
 'Coal PC':

## 4. Do LCIA

As a simplification, if there are several ecoinvent activities per REMIND one, we use an averaged demand in the LCA.

In [9]:
methods =  [
    ('ReCiPe Midpoint (H) V1.13', 'agricultural land occupation', 'ALOP'),
    ('ReCiPe Midpoint (H) V1.13', 'climate change', 'GWP100'),
    ('ReCiPe Midpoint (H) V1.13', 'fossil depletion', 'FDP'),
    ('ReCiPe Midpoint (H) V1.13', 'freshwater ecotoxicity', 'FETPinf'),
    ('ReCiPe Midpoint (H) V1.13', 'freshwater eutrophication', 'FEP'),
    ('ReCiPe Midpoint (H) V1.13', 'human toxicity', 'HTPinf'),
    ('ReCiPe Midpoint (H) V1.13', 'ionising radiation', 'IRP_HE'),
    ('ReCiPe Midpoint (H) V1.13', 'marine ecotoxicity', 'METPinf'),
    ('ReCiPe Midpoint (H) V1.13', 'marine eutrophication', 'MEP'),
    ('ReCiPe Midpoint (H) V1.13', 'metal depletion', 'MDP'),
    ('ReCiPe Midpoint (H) V1.13', 'natural land transformation', 'NLTP'),
    ('ReCiPe Midpoint (H) V1.13', 'ozone depletion', 'ODPinf'),
    ('ReCiPe Midpoint (H) V1.13', 'particulate matter formation', 'PMFP'),
    ('ReCiPe Midpoint (H) V1.13', 'photochemical oxidant formation', 'POFP'),
    ('ReCiPe Midpoint (H) V1.13', 'terrestrial acidification', 'TAP100'),
    ('ReCiPe Midpoint (H) V1.13', 'terrestrial ecotoxicity', 'TETPinf'),
    ('ReCiPe Midpoint (H) V1.13', 'urban land occupation', 'ULOP'),
    ('ReCiPe Midpoint (H) V1.13', 'water depletion', 'WDP')
]

In [1]:
from premise.geomap import Geomap

geomap = Geomap("remind")

In [44]:
geomap.iam_to_ecoinvent_location("JPN")

['JP']

In [26]:
eidb = bw.Database("ecoinvent_remind_default_2030")

In [22]:
remind_regions = list(set(iam_data.production_volumes.coords["region"].values) - {"World"})
remind_regions

['USA',
 'MEA',
 'JPN',
 'NEU',
 'SSA',
 'IND',
 'CHA',
 'REF',
 'EUR',
 'LAM',
 'CAZ',
 'OAS']

In [48]:
# collect activities
import itertools

act_dict = {i: [] for i in itertools.product(mapping.keys(), remind_regions)}

for act in eidb:
    for reg in remind_regions:
        ei_regions = geomap.iam_to_ecoinvent_location(reg) + [reg]
        for k, v in mapping.items():
            for name in v:
                if name in act["name"] and act["location"] in ei_regions:
                    if act["unit"] == "kilowatt hour":
                        act_dict[(k, reg)].append(act)

In [56]:
fallbacks = {k: [] for k in mapping.keys()}

for act in eidb:
    for k, v in mapping.items():
        for name in v:
            if name in act["name"] and act["location"] in ["RoW", "GLO"]:
                if act["unit"] == "kilowatt hour":
                    fallbacks[k].append(act)

fallbacks

{'Biomass CHP': ['heat and power co-generation, wood chips, 6667 kW' (kilowatt hour, RoW, None),
  'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014' (kilowatt hour, RoW, None),
  'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014' (kilowatt hour, RoW, None)],
 'Biomass CHP CCS': [],
 'Biomass ST': [],
 'Biomass IGCC CCS': [],
 'Biomass IGCC': [],
 'Coal PC': ['electricity production, hard coal' (kilowatt hour, RoW, None),
  'electricity production, hard coal, at coal mine power plant' (kilowatt hour, RoW, None),
  'electricity production, hard coal, at coal mine power plant' (kilowatt hour, RoW, None),
  'electricity production, lignite' (kilowatt hour, RoW, None)],
 'Coal IGCC': [],
 'Coal PC CCS': [],
 'Coal IGCC CCS': [],
 'Coal CHP': ['heat and power co-generation, lignite' (kilowatt hour, RoW, None),
  'heat and power co-generation, hard coal' (kilowatt hour, RoW, None)],
 'Coal CHP CCS': [],
 'Gas OC': ['electricity production, natu

In [60]:
for i, v in act_dict.items():
    if len(v) == 0:
        k = i[0]
        act_dict[i] = fallbacks[k]

for i, v in act_dict.items():
    if len(v) == 0:
        act_dict[i] = act_dict[(i[0], "USA")]

In [63]:
lcia_scores = {}

counter = 0
for k in mapping.keys():
    for reg in remind_regions:
        activities = act_dict[(k, reg)]
        N = len(activities)
        demand = {a: 1/N for a in activities}
        lca = bw.LCA(demand)
        lca.lci()

        scores = {}
        for m in methods:
            lca.switch_method(m)
            lca.lcia()
            scores[m[1]] = lca.score

        lcia_scores[(k, reg)] = scores
        counter += 1
        print("{} / {}".format(counter, len(act_dict)))

1 / 348
2 / 348
3 / 348
4 / 348
5 / 348
6 / 348
7 / 348
8 / 348
9 / 348
10 / 348
11 / 348
12 / 348
13 / 348
14 / 348
15 / 348
16 / 348
17 / 348
18 / 348
19 / 348
20 / 348
21 / 348
22 / 348
23 / 348
24 / 348
25 / 348
26 / 348
27 / 348
28 / 348
29 / 348
30 / 348
31 / 348
32 / 348
33 / 348
34 / 348
35 / 348
36 / 348
37 / 348
38 / 348
39 / 348
40 / 348
41 / 348
42 / 348
43 / 348
44 / 348
45 / 348
46 / 348
47 / 348
48 / 348
49 / 348
50 / 348
51 / 348
52 / 348
53 / 348
54 / 348
55 / 348
56 / 348
57 / 348
58 / 348
59 / 348
60 / 348
61 / 348
62 / 348
63 / 348
64 / 348
65 / 348
66 / 348
67 / 348
68 / 348
69 / 348
70 / 348
71 / 348
72 / 348
73 / 348
74 / 348
75 / 348
76 / 348
77 / 348
78 / 348
79 / 348
80 / 348
81 / 348
82 / 348
83 / 348
84 / 348
85 / 348
86 / 348
87 / 348
88 / 348
89 / 348
90 / 348
91 / 348
92 / 348
93 / 348
94 / 348
95 / 348
96 / 348
97 / 348
98 / 348
99 / 348
100 / 348
101 / 348
102 / 348
103 / 348
104 / 348
105 / 348
106 / 348
107 / 348
108 / 348
109 / 348
110 / 348
111 / 34

In [127]:
EJ_to_kWh = 10e12 / 3.6

sectors = list(iam_data.production_volumes.coords["variables"].values)

lcia_totals = {}

for i, scores in lcia_scores.items():
    k = i[0]
    reg = i[1]
    if k in sectors:
        volume = float(iam_data.production_volumes.sel(region=reg, variables=k, year=2030)) 
        lcia_totals[i] = {m: v * EJ_to_kWh * volume for m, v in scores.items()}

totals = pd.DataFrame(lcia_totals).T.stack()
idx = totals.index
new_idx = pd.MultiIndex.from_tuples(idx.to_list(), names=["variable", "region", "midpoint"])
totals = pd.Series(totals.values, index=new_idx).to_xarray()
totals

<xarray.DataArray (variable: 20, region: 12, midpoint: 18)>
array([[[6.90489217e+06, 5.30764336e+07, 1.46741017e+07, ...,
         9.96507371e+04, 2.75104001e+06, 1.05182629e+06],
        [1.32840981e+09, 1.84503370e+10, 5.41019304e+09, ...,
         3.47948205e+07, 8.44590855e+08, 3.05975936e+08],
        [1.07968343e+10, 6.68089821e+10, 2.05419065e+10, ...,
         1.52910856e+08, 4.19797095e+09, 8.13799495e+08],
        ...,
        [1.85928169e+09, 2.46744634e+10, 6.62701863e+09, ...,
         4.45832185e+07, 1.15576713e+09, 4.84435237e+08],
        [1.76036650e+04, 2.47909146e+05, 7.01099551e+04, ...,
         4.79345203e+02, 1.20306086e+04, 4.53750334e+03],
        [1.38668217e+08, 1.75224930e+09, 5.01042015e+08, ...,
         3.57204332e+06, 8.19484592e+07, 3.08598811e+07]],

       [[1.22362123e+09, 8.71879639e+09, 3.11064263e+09, ...,
         1.10496383e+07, 5.54661395e+08, 5.21183005e+07],
        [2.09796316e+09, 2.67698120e+10, 9.60840403e+09, ...,
         3.69894155e+07, 1.46915019e+09, 1.42872574e+08],
        [1.39436415e+10, 8.44439797e+10, 2.82406497e+10, ...,
         1.76350348e+08, 5.64088719e+09, 6.35186764e+08],
...
        [2.57557556e+06, 5.62489724e+07, 1.65291333e+07, ...,
         1.99797852e+04, 1.07538092e+06, 8.52249201e+05],
        [9.71046852e+06, 2.12070608e+08, 6.23183535e+07, ...,
         7.53280465e+04, 4.05441516e+06, 3.21316104e+06],
        [8.33765433e+07, 1.82089198e+09, 5.35081174e+08, ...,
         6.46785695e+05, 3.48122359e+07, 2.75890149e+07]],

       [[7.42833072e+08, 1.40700664e+10, 4.34004924e+09, ...,
         5.72997911e+06, 1.45793632e+09, 1.98124364e+08],
        [1.20257017e+10, 2.21922378e+11, 6.80390220e+10, ...,
         9.18183161e+07, 2.20583135e+10, 3.10887157e+09],
        [5.47204955e+09, 9.91273556e+10, 3.02890336e+10, ...,
         4.24257243e+07, 9.70126382e+09, 1.38840085e+09],
        ...,
        [3.18604123e+08, 6.48059665e+09, 2.03026625e+09, ...,
         2.54666194e+06, 7.46993227e+08, 9.27244046e+07],
        [9.67487324e+08, 1.78910228e+10, 5.48782740e+09, ...,
         7.39270646e+06, 1.78442984e+09, 2.50744392e+08],
        [4.41226602e+09, 8.15927507e+10, 2.50274642e+10, ...,
         3.37147441e+07, 8.13796618e+09, 1.14353019e+09]]])
Coordinates:
  * variable  (variable) object 'Biomass CHP' 'Biomass IGCC' ... 'Wind Onshore'
  * region    (region) object 'CAZ' 'CHA' 'EUR' 'IND' ... 'REF' 'SSA' 'USA'
  * midpoint  (midpoint) object 'agricultural land occupation' ... 'water dep...

In [128]:
monetization_factors = pd.read_excel("arendt-et-al.xlsx")
monetization_factors

,Method,Impact category,unit,min,central,max,note
0,Stepwise,Global warming,2019€/kg CO2-eq,NaN,0.107282,NaN,NaN
1,Stepwise,Acidification,2019€/kg SO2-eq,NaN,0.193880,NaN,NaN
2,Stepwise,Stratospheric ozone depletion,2019€/kg CFC-11-eq,NaN,129.255060,NaN,NaN
3,Stepwise,Freshwater eutrophication,2019€/kg PO4-eq,NaN,1.551061,NaN,NaN
4,Stepwise,Marine eutrophication,2019€/kg NO3-eq,NaN,0.572000,NaN,NaN
...,...,...,...,...,...,...,...
157,EVR,Particulate matter,2019€/kg PM 2.5-eq,NaN,36.038000,NaN,NaN
158,EVR,Land use,2019€/m2/yr,0.0000,NaN,6.170,"min: natural forest, max: heterogeneous agricu..."
159,EVR,Land transformation,2019€/m2,0.1544,NaN,7.720,"min: countries with low biodiversity, max: cou..."
160,EVR,Soil organic matter,2019€/kg C-eq,NaN,NaN,NaN,NaN


In [129]:
stepwise_factors = monetization_factors[monetization_factors["Method"] == "Stepwise"]
stepwise_factors = xr.DataArray(np.array(stepwise_factors["central"]), coords=[stepwise_factors["Impact category"]], dims=["Impacts"])
stepwise_factors = stepwise_factors.astype(float)

Define conversion factors between ReCiPe midpoints and Stepwise categories.

In [131]:
midpoints = [m[1] for m in methods]

conv_factors = xr.DataArray(
    np.zeros((18, 18)), 
    coords=[stepwise_factors.coords["Impacts"], midpoints],
    dims = ["Impacts", "midpoint"]
    )

In [132]:
conv_factors.loc["Global warming", "climate change"] = 1
conv_factors.loc["Water use", "water depletion"] = 1
conv_factors.loc["Stratospheric ozone depletion", "ozone depletion"] = 1
conv_factors.loc["Acidification", "terrestrial acidification"] = 1
conv_factors.loc["Terrestrial ecotoxicity", "terrestrial ecotoxicity"] = 1
conv_factors.loc["Human toxicity", "human toxicity"] = 1.81e-07 * 11.5
conv_factors.loc["Freshwater eutrophication", "freshwater eutrophication"] = 3.03
conv_factors.loc["Marine eutrophication", "marine eutrophication"] = 4.34
conv_factors.loc["Photochemical oxidation", "photochemical oxidant formation"] = 0.496
conv_factors.loc["Mineral resources", "metal depletion"] = 1 # couldn't find a conversion
conv_factors.loc["Fossil resources", "fossil depletion"] = 44.5 # heating value of oil in MJ/kg
conv_factors.loc["Ionizing radiation", "ionising radiation"] = 1
conv_factors.loc["Freshwater ecotoxicity", "freshwater ecotoxicity"] = 1
conv_factors.loc["Marine ecotoxicity", "marine ecotoxicity"] = 1
conv_factors.loc["Particulate matter", "particulate matter formation"] = 0.28
conv_factors.loc["Land use", "agricultural land occupation"] = 1
conv_factors.loc["Land use", "urban land occupation"] = 1
conv_factors.loc["Land transformation", "natural land transformation"] = 1

In [133]:
impacts = conv_factors.dot(totals)
impacts

<xarray.DataArray (Impacts: 18, variable: 20, region: 12)>
array([[[ 5.30764336e+07,  1.84503370e+10,  6.68089821e+10, ...,
          2.46744634e+10,  2.47909146e+05,  1.75224930e+09],
        [ 8.71879639e+09,  2.67698120e+10,  8.44439797e+10, ...,
          4.60748903e+05,  7.55407112e+08,  6.15886162e+10],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        ...,
        [ 1.73765971e+10,  3.50960779e+11,  1.17529555e+11, ...,
          8.12051007e+09,  1.03831181e+10,  1.17751847e+11],
        [ 9.24288670e+08,  9.16424555e+09,  1.81798826e+10, ...,
          5.62489724e+07,  2.12070608e+08,  1.82089198e+09],
        [ 1.40700664e+10,  2.21922378e+11,  9.91273556e+10, ...,
          6.48059665e+09,  1.78910228e+10,  8.15927507e+10]],

       [[ 1.35929419e+06,  4.25389528e+08,  9.22771807e+08, ...,
          6.47807155e+08,  6.01001771e+03,  3.87878783e+07],
        [ 5.60515975e+07,  1.76599120e+08,  5.93500448e+08, ...,
          3.08172227e+03,  5.31174500e+06,  3.67037200e+08],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
...
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[ 1.05182629e+06,  3.05975936e+08,  8.13799495e+08, ...,
          4.84435237e+08,  4.53750334e+03,  3.08598811e+07],
        [ 5.21183005e+07,  1.42872574e+08,  6.35186764e+08, ...,
          2.48114612e+03,  4.22347090e+06,  3.76219995e+08],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        ...,
        [ 4.13851902e+08,  8.11572508e+09,  2.83051515e+09, ...,
          1.70913300e+08,  2.34237613e+08,  2.47833653e+09],
        [ 1.40042430e+07,  1.39290825e+08,  2.77024177e+08, ...,
          8.52249201e+05,  3.21316104e+06,  2.75890149e+07],
        [ 1.98124364e+08,  3.10887157e+09,  1.38840085e+09, ...,
          9.27244046e+07,  2.50744392e+08,  1.14353019e+09]]])
Coordinates:
  * Impacts   (Impacts) object 'Global warming' 'Acidification' ... 'Water use'
  * variable  (variable) object 'Biomass CHP' 'Biomass IGCC' ... 'Wind Onshore'
  * region    (region) object 'CAZ' 'CHA' 'EUR' 'IND' ... 'REF' 'SSA' 'USA'

In [139]:
(stepwise_factors * impacts)

<xarray.DataArray (Impacts: 18, variable: 20, region: 12)>
array([[[5.69412987e+06, 1.97938346e+09, 7.16738098e+09, ...,
         2.64711831e+09, 2.65961138e+04, 1.87984278e+08],
        [9.35367273e+08, 2.87191086e+09, 9.05929344e+09, ...,
         4.94299242e+04, 8.10413569e+07, 6.60733126e+09],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [1.86419082e+09, 3.76516679e+10, 1.26087701e+10, ...,
         8.71182101e+08, 1.11391853e+09, 1.26326179e+10],
        [9.91592571e+07, 9.83155815e+08, 1.95036865e+09, ...,
         6.03448522e+06, 2.27512948e+07, 1.95348382e+08],
        [1.50946060e+09, 2.38082093e+10, 1.06345509e+10, ...,
         6.95249407e+08, 1.91937929e+09, 8.75340876e+09]],

       [[2.63539957e+05, 8.24745217e+07, 1.78906998e+08, ...,
         1.25596851e+08, 1.16522223e+03, 7.52019384e+06],
        [1.08672837e+07, 3.42390374e+07, 1.15067867e+08, ...,
         5.97484314e+02, 1.02984112e+06, 7.11611723e+07],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
...
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan]],

       [[           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        ...,
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan]]])
Coordinates:
  * Impacts   (Impacts) object 'Global warming' 'Acidification' ... 'Water use'
  * variable  (variable) object 'Biomass CHP' 'Biomass IGCC' ... 'Wind Onshore'
  * region    (region) object 'CAZ' 'CHA' 'EUR' 'IND' ... 'REF' 'SSA' 'USA'

In [138]:
(stepwise_factors * impacts).sum(axis=0)

<xarray.DataArray (variable: 20, region: 12)>
array([[2.91907945e+07, 1.05546597e+10, 2.77231638e+10, 2.59478242e+09,
        1.15202014e+05, 1.06233862e+09, 2.44801937e+05, 1.62114745e+09,
        1.05268242e+08, 1.69807114e+10, 1.62660254e+05, 8.77727759e+08],
       [3.35555403e+09, 9.87780949e+09, 2.90815521e+10, 8.72227865e+09,
        3.54320926e+10, 3.48815228e+10, 1.57063513e+06, 1.02244851e+09,
        1.83875538e+09, 1.71954506e+05, 2.85503989e+08, 2.22260696e+10],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.74834379e+09, 7.49409533e+10, 3.29547932e+11, 4.05902503e+06,
        1.00390798e+11, 4.79547509e+10, 2.48144915e+06, 2.65238305e+08,
        3.55055871e+10, 1.62744842e+12, 2.13632896e+06, 3.67769031e+08],
       [1.23933718e+06, 2.74040494e+11, 1.22197978e+06, 3.15110073e+10,
        1.39951599e+11, 1.31960981e+09, 9.76589551e+05, 3.51969213e+09,
        1.53849358e+10, 1.81089700e+09, 9.52190502e+05, 6.47851989e+09],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.57915302e+11, 9.35971032e+12, 6.46799485e+11, 4.90907210e+12,
        2.73784471e+11, 1.83875057e+11, 4.15275645e+10, 8.50575059e+10,
...
        2.28376992e+09, 2.42477969e+11, 4.00399996e+09, 6.84389169e+09,
        3.64823809e+10, 1.21422367e+10, 1.79916393e+10, 1.16827554e+10],
       [3.19972055e+09, 3.88852830e+10, 3.75077104e+10, 7.35549523e+09,
        8.06912489e+09, 1.72210598e+09, 1.61128370e+09, 1.04871766e+09,
        9.36667437e+09, 1.38094644e+10, 4.82749771e+08, 3.35491653e+10],
       [3.77864109e+09, 2.53702932e+10, 2.92426823e+10, 7.75032535e+09,
        2.61585747e+10, 4.74508982e+10, 7.44540288e+10, 1.19105730e+09,
        4.04685202e+10, 1.03823386e+09, 4.29913920e+09, 3.01703877e+10],
       [1.46488302e+07, 1.66416251e+08, 6.74149159e+08, 1.65737314e+09,
        2.33197293e+04, 1.26807025e+09, 1.35038556e+09, 9.63771098e+06,
        4.56566986e+07, 1.05544762e+09, 4.90504390e+08, 2.97469158e+09],
       [6.47112403e+09, 1.45174419e+11, 4.24717607e+10, 3.67660881e+10,
        7.76623399e+09, 2.00079668e+10, 1.04834633e+10, 5.57195896e+09,
        3.15381075e+10, 3.29497673e+09, 4.44891361e+09, 4.77789684e+10],
       [2.68389100e+08, 2.66643837e+09, 5.28595746e+09, 2.67032940e+08,
        1.83029333e+08, 4.30059717e+08, 1.97611809e+07, 8.90706551e+07,
        8.61062525e+08, 1.63332209e+07, 6.15797223e+07, 5.28739099e+08],
       [4.76014150e+09, 7.56779488e+10, 3.38752590e+10, 1.82912816e+10,
        9.44529072e+08, 1.06581416e+10, 1.04600665e+09, 4.79532106e+09,
        7.65109522e+09, 2.14443086e+09, 6.09716300e+09, 2.78063644e+10]])
Coordinates:
  * variable  (variable) object 'Biomass CHP' 'Biomass IGCC' ... 'Wind Onshore'
  * region    (region) object 'CAZ' 'CHA' 'EUR' 'IND' ... 'REF' 'SSA' 'USA'